In [422]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import wikipedia
import json
import xml.etree.ElementTree as ET
import time

In [423]:
# A continuación se pregunta al usuario por el identificador de la clase del cual se quiere generar un diagrama
print("¿Cual es el identificador de la clase del cual se quiere generar un diagrama?")
clase = input()
print("¿Cual es el idioma en el que quieres generar el diagrama (ES:español, EN:ingles, ...)?")
idioma = input()

¿Cual es el identificador de la clase del cual se quiere generar un diagrama?
Q1004
¿Cual es el idioma en el que quieres generar el diagrama (ES:español, EN:ingles, ...)?
ES


In [424]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url)    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

#Se genera la consulta para obtener el numero de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [425]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q91879,literal,Q91879,NaN
1,uri,http://www.wikidata.org/entity/Q91892,literal,Q91892,NaN
2,uri,http://www.wikidata.org/entity/Q91912,literal,Q91912,NaN
3,uri,http://www.wikidata.org/entity/Q91943,literal,Q91943,NaN
4,uri,http://www.wikidata.org/entity/Q92191,literal,Q92191,NaN


In [426]:
#Se obtiene el numero de instancias de la clase de entrada
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

4592

In [427]:
#Se genera la consulta para obtener las propiedades de la clase de entrada
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [428]:
#Se confirma que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P407,literal,http://www.wikidata.org/prop/direct/P407
2,uri,http://www.wikidata.org/prop/direct/P495,literal,http://www.wikidata.org/prop/direct/P495
3,uri,http://www.wikidata.org/prop/direct/P571,literal,http://www.wikidata.org/prop/direct/P571
4,uri,http://www.wikidata.org/prop/direct/P580,literal,http://www.wikidata.org/prop/direct/P580


In [429]:
#Se genera la consulta para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

#Se listan las propiedades de salida de la clase y se le asignan a la variable "lista_propiedades"
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)
print(len(lista_propiedades))

227


In [434]:
lista_resultado_propiedades_clases = []
#Para cada propiedad de la lista:
for prop in lista_propiedades:
    
    #Se obtienen de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
    #Aquí se genera la consulta. Esta se hace para ver si la propiedad con la que se esta tratando es object property o no.
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    
    #Se obtiene el valor booleano del xml string, siendo este "true" en caso afirmativo
    es_OP = string[105:109]
    
    #En caso de que la variable es_OP sea "true", es decir la propiedad sea object propery:
    if es_OP == "true":
        
        #Se genera la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad.
        #Se realiza esta consulta para todas las object properties encontradas anteriormente.
        sparql1 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql1.setQuery("""
        SELECT DISTINCT ?claseInicial ?claseInicialLabel ?prop ?propLabel ?Clase ?ClaseLabel
        WHERE {
        #occupation (P106) must be Formula 1 driver (Q10841764)
        ?claseInicial wdt:P31 wd:"""+clase+""" .
        ?claseInicial wdt:"""+prop +""" ?prop .
        ?prop wdt:P31 ?Clase .
        #to obtain labels in spanish:
        SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
        }  
        """)
        sparql1.setReturnFormat(JSON)
        results = sparql1.query().convert()
        
        #Se añade a la lista "lista_clases_para_una_propiedad_completo" todas las clases que resultan de la consulta anterior. 
        lista_clases_para_una_propiedad_completo = []
        for result in results["results"]["bindings"]:
            lista_clases_para_una_propiedad_completo.append(result["Clase"]["value"])
        
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_clases_para_una_propiedad = []
        for ele in lista_clases_para_una_propiedad_completo:
            ele = ele[31:]
            lista_clases_para_una_propiedad.append(ele)
        
        #Se tiene la lista "lista_clases_para_una_propiedad" en la cual se encuentran todas las clases de la propiedad 
        #estudiada.
        
        #Se obtienen las value type constraint de las propiedades. La consulta siguiente las obtiene.
        sparql2 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql2.setQuery("""SELECT ?Property_ ?Property_Label ?Property_Description ?class_ ?class_Label ?relation_ ?relation_Label
        WHERE {
        ?Property_ wikibase:directClaim wdt:"""+prop+""" .
        ?Property_ p:P2302 ?constraint_statement .
        ?constraint_statement ps:P2302 wd:Q21510865 .
        OPTIONAL {?constraint_statement pq:P2308 ?class_ .}
        OPTIONAL {?constraint_statement pq:P2309 ?relation_ .}

        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } 
        """)
        sparql2.setReturnFormat(JSON)
        results = sparql2.query().convert()

        #En la lista "lista_value_type_constraints_completa" se almacenan todas las value type constraint de la propiedad 
        #estudiada.
        lista_value_type_constraints_completa = []
        for result in results["results"]["bindings"]:
            lista_value_type_constraints_completa.append(result["class_"]["value"])
            
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_value_type_constraints = []
        for ele in lista_value_type_constraints_completa:
            ele = ele[31:]
            lista_value_type_constraints.append(ele)
        
        #Se tiene la lista "lista_value_type_constraints" con las value type constraint de la propiedad estudiada.
        if (lista_clases_para_una_propiedad != []) & (lista_value_type_constraints != []):
            #Se guardan en la lista "lista_clases_a_representar" aquellos identificadores que esten en ambas listas.
            lista_clases_a_representar = []
            for ele_VTC in lista_value_type_constraints:
                for ele_CPUP in lista_clases_para_una_propiedad:
                    if ele_VTC == ele_CPUP:
                        lista_clases_a_representar.append(ele_VTC)
            
            #Se tiene la lista "lista_clases_a_representar" con las clases que hay que representar para la propiedad estudiada.
            
            #Se eliminan los duplicados
            if lista_clases_a_representar != []:             
                result = []
                for ele in lista_clases_a_representar:
                    if ele not in result:
                        result.append(ele)
                        
                #Se añade a la lista final el valor de la propiedad
                lista_resultado_propiedades_clases.append(prop)
                
                result2 = result
                result3 = result
                #Se eliminan las clases que sean subclases de otra clase de la misma lista
                if len(result) != 1:
                    for ele in result:
                        #Se obtienen las subclases de la clase ele
                        sparql3 = SPARQLWrapper("https://query.wikidata.org/sparql")
                        sparql3.setQuery("""SELECT ?s ?desc
                        WHERE{
                        ?s wdt:P279+ wd:"""+ele+""" .
                        OPTIONAL {
                        ?s rdfs:label ?desc filter (lang(?desc) = "es").}
                        }
                        """)
                        sparql3.setReturnFormat(JSON)
                        results = sparql3.query().convert()

                        #En la lista "lista_value_type_constraints_completa" se almacenan todas las value type constraint de 
                        #la propiedad estudiada.
                        lista_subclases_de_ele_completa = []
                        for result in results["results"]["bindings"]:
                            lista_subclases_de_ele_completa.append(result["s"]["value"])
                           
                        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador 
                        #de la clase.
                        lista_subclases_de_ele = []
                        for ele2 in lista_subclases_de_ele_completa:
                            ele2 = ele2[31:]
                            lista_subclases_de_ele.append(ele2)
                          
                        #Se comparan todos los elementos de la lista "result2" con los de la lista "lista_subclase_de_ele" y 
                        #se eliman de la lista final si estos son iguales.
                        for ele4 in result2:
                            for ele3 in lista_subclases_de_ele:
                                if ele4 == ele3:
                                    result3.remove(ele4)
                
                #Se añaden las listas con las clases a representar en la lista final.
                lista_resultado_propiedades_clases.append(result3)
    
                
#Se imprime la lista resultado final con las propiedades a representar y sus respectivas clases
print(lista_resultado_propiedades_clases)

['P495', ['Q6256', 'Q7275', 'Q3024240'], 'P674', ['Q95074'], 'P6338', ['Q5'], 'P50', ['Q5', 'Q43229'], 'P110', ['Q5'], 'P123', ['Q11032', 'Q2085381'], 'P136', ['Q483394'], 'P840', ['Q82794', 'Q3895768'], 'P910', ['Q4167836'], 'P941', ['Q5'], 'P17', ['Q6256', 'Q43702'], 'P170', ['Q5'], 'P1434', ['Q559618'], 'P1080', ['Q559618'], 'P2354', ['Q13406463'], 'P166', ['Q618779'], 'P5008', ['Q16695773'], 'P8345', ['Q196600'], 'P747', ['Q3331189'], 'P161', ['Q5'], 'P58', ['Q5'], 'P449', ['Q15265344'], 'P462', ['Q1075'], 'P57', ['Q5'], 'P106', ['Q28640', 'Q17305127'], 'P767', ['Q5'], 'P676', ['Q5'], 'P736', ['Q5'], 'P86', ['Q5'], 'P1411', ['Q618779'], 'P98', ['Q5'], 'P5202', ['Q5'], 'P655', ['Q5'], 'P793', ['Q1190554'], 'P2679', ['Q5'], 'P135', ['Q2198855'], 'P175', ['Q215380'], 'P7936', ['Q815823'], 'P1877', ['Q5'], 'P275', ['Q79719'], 'P5769', ['Q5'], 'P112', ['Q5'], 'P437', ['Q81941037'], 'P488', ['Q5'], 'P641', ['Q31629']]


In [435]:
#Se generan las listas "lista_varias_propiedades_una_clase" y "lista_varias_clases_una_propiedad".
#En la primera aparecen todas las propiedades que apuntan a una sola clase.
#En la segunda aparecen todas las clases a las que apunta una sola propiedad.
result = []
result2 = []
for ele in range(0,len(lista_resultado_propiedades_clases)):
    if lista_resultado_propiedades_clases[ele] not in result:
        result.append(lista_resultado_propiedades_clases[ele])
    else:
        if lista_resultado_propiedades_clases[ele] not in result2:
            result2.append(lista_resultado_propiedades_clases[ele])
            m=[i for i,x in enumerate(lista_resultado_propiedades_clases) if x==lista_resultado_propiedades_clases[ele]]
            for i in m:
                result2.append(lista_resultado_propiedades_clases[i-1])

lista_varias_propiedades_una_clase = result2
lista_varias_clases_una_propiedad = lista_resultado_propiedades_clases

def eliminar_valores_de_una_lista(the_list, val):
    return [value for value in the_list if value != val]

for x in lista_varias_clases_una_propiedad:
    for y in lista_varias_propiedades_una_clase:
        if x == y:
            lista_varias_clases_una_propiedad = eliminar_valores_de_una_lista(lista_varias_clases_una_propiedad, y)

print(lista_varias_propiedades_una_clase)
print("\n")
print(lista_varias_clases_una_propiedad)

[['Q5'], 'P6338', 'P110', 'P941', 'P170', 'P161', 'P58', 'P57', 'P767', 'P676', 'P736', 'P86', 'P98', 'P5202', 'P655', 'P2679', 'P1877', 'P5769', 'P112', 'P488', ['Q559618'], 'P1434', 'P1080', ['Q618779'], 'P166', 'P1411']


['P495', ['Q6256', 'Q7275', 'Q3024240'], 'P674', ['Q95074'], 'P50', ['Q5', 'Q43229'], 'P123', ['Q11032', 'Q2085381'], 'P136', ['Q483394'], 'P840', ['Q82794', 'Q3895768'], 'P910', ['Q4167836'], 'P17', ['Q6256', 'Q43702'], 'P2354', ['Q13406463'], 'P5008', ['Q16695773'], 'P8345', ['Q196600'], 'P747', ['Q3331189'], 'P449', ['Q15265344'], 'P462', ['Q1075'], 'P106', ['Q28640', 'Q17305127'], 'P793', ['Q1190554'], 'P135', ['Q2198855'], 'P175', ['Q215380'], 'P7936', ['Q815823'], 'P275', ['Q79719'], 'P437', ['Q81941037'], 'P641', ['Q31629']]


In [436]:
#Se busca la etiqueta de cada uno de los elementos de las dos listas y se superponen al identificador anterior
for pos in range(0,len(lista_varias_propiedades_una_clase)):
    res = isinstance(lista_varias_propiedades_una_clase[pos], str) 
    #Para las propiedades
    if str(res) == "True":
        #Se obtienen las subclases de la clase ele
        sparql4 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql4.setQuery("""SELECT  *
        WHERE {
        wd:"""+lista_varias_propiedades_una_clase[pos]+""" rdfs:label ?label .
        FILTER (langMatches( lang(?label),'"""+idioma+"""' ) )
        } 
        LIMIT 1
        """)
        sparql4.setReturnFormat(JSON)
        results = sparql4.query().convert()
        lista_label = []
        for result in results["results"]["bindings"]:
            lista_label.append(result["label"]["value"])
        for ele2 in lista_label:
            label = ele2
        label = label + " ("+lista_varias_propiedades_una_clase[pos]+")"
        lista_varias_propiedades_una_clase[pos] = label
    #Para las listas de clases
    else:
        for ele in range(0,len(lista_varias_propiedades_una_clase[pos])):
           #Se obtienen las subclases de la clase ele
            sparql4 = SPARQLWrapper("https://query.wikidata.org/sparql")
            sparql4.setQuery("""SELECT  *
            WHERE {
            wd:"""+lista_varias_propiedades_una_clase[pos][ele]+""" rdfs:label ?label .
            FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
            } 
            LIMIT 1
            """)
            sparql4.setReturnFormat(JSON)
            results = sparql4.query().convert()
            lista_label = []
            for result in results["results"]["bindings"]:
                lista_label.append(result["label"]["value"])
            for ele2 in lista_label:
                label = ele2
            label = label + " ("+lista_varias_propiedades_una_clase[pos][ele]+")"
            lista_varias_propiedades_una_clase[pos][ele] = label 

for pos in range(0,len(lista_varias_clases_una_propiedad)):
    res = isinstance(lista_varias_clases_una_propiedad[pos], str) 
    #Para las propiedades
    if str(res) == "True":
        #Se obtienen las subclases de la clase ele
        sparql4 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql4.setQuery("""SELECT  *
        WHERE {
        wd:"""+lista_varias_clases_una_propiedad[pos]+""" rdfs:label ?label .
        FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
        } 
        LIMIT 1
        """)
        sparql4.setReturnFormat(JSON)
        results = sparql4.query().convert()
        lista_label = []
        for result in results["results"]["bindings"]:
            lista_label.append(result["label"]["value"])
        for ele2 in lista_label:
            label = ele2
        label = label + " ("+lista_varias_clases_una_propiedad[pos]+")"
        lista_varias_clases_una_propiedad[pos] = label
    #Para las listas de clases
    else:
        for ele in range(0,len(lista_varias_clases_una_propiedad[pos])):
           #Se obtienen las subclases de la clase ele
            sparql5 = SPARQLWrapper("https://query.wikidata.org/sparql")
            sparql5.setQuery("""SELECT  *
            WHERE {
            wd:"""+lista_varias_clases_una_propiedad[pos][ele]+""" rdfs:label ?label .
            FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
            } 
            LIMIT 1
            """)
            sparql5.setReturnFormat(JSON)
            results = sparql5.query().convert()
            lista_label = []
            for result in results["results"]["bindings"]:
                lista_label.append(result["label"]["value"])
            for ele2 in lista_label:
                label = ele2
            label = label + " ("+lista_varias_clases_una_propiedad[pos][ele]+")"
            lista_varias_clases_una_propiedad[pos][ele] = label 
        
print(lista_varias_propiedades_una_clase)
print("\n")
print(lista_varias_clases_una_propiedad)

[['ser humano (Q5)'], 'colorista (P6338)', 'ilustrador (P110)', 'inspirado por (P941)', 'creador (P170)', 'miembro del reparto (P161)', 'guionista (P58)', 'director (P57)', 'colaborador (P767)', 'letra de (P676)', 'artista de la portada (P736)', 'compositor (P86)', 'editor (P98)', 'adaptación de (P5202)', 'traductor (P655)', 'autor del prefacio (P2679)', 'inspirado en la obra de (P1877)', 'redactor en jefe (P5769)', 'fundador (P112)', 'presidente (P488)', ['universo de ficción (Q559618)'], 'describe el universo ficticio (P1434)', 'del universo ficticio (P1080)', ['distinción (Q618779)'], 'premio recibido (P166)', 'nominado a (P1411)']


['país de origen (P495)', ['país (Q6256)', 'Estado (Q7275)', 'Estado desaparecido (Q3024240)'], 'personajes (P674)', ['personaje de ficción (Q95074)'], 'autor (P50)', ['ser humano (Q5)', 'organización (Q43229)'], 'editorial (P123)', ['periódico (Q11032)', 'editorial (Q2085381)'], 'género (P136)', ['género (Q483394)'], 'lugar de la narración (P840)', ['r

In [437]:
lista_a_no_representar = ['P646','P957','P1417','P3265','P3417','P3589','P7818','P212','P1274','P1296','P2638','P3365','P2671',
                          'P6839','P244','P373','P2003','P2013','P2163','P3984','P4073','P214','P227','P1267','P7859','P268',
                          'P4485','P8873','P6366','P345','P480','P648','P724','P905','P1258','P1265','P1970','P2334','P2465',
                          'P2508','P2509','P2529','P2603','P2704','P3107','P3143','P3138','P3212','P3302','P3704','P3868',
                          'P3933','P4277','P4282','P4529','P4947','P5032','P5786','P5849','P6127','P6133','P8033','P1392',
                          'P1092','P5396','P2969','P8383','P7363','P236','P243','P2002','P5331','P5099','P3943','P6262','P8406',
                          'P6947','P8687','P1273','P8179','P2397','P2635','P6981','P3959','P5749','P7704','P2687','P5733',
                          'P1922','P3132','P6164','P8644','P1085','P3962','P1235','P3631','P6623','P675','P1984','P4087',
                          'P5905','P1985','P5357','P2581','P3222','P3219','P4342','P5019','P7818','P8349','P244','P1036','P1430',
                          'P7772','P8408','P7982','P2209','P2209','P9100','P935','P349','P691','P1014','P1051','P1245','P1417',
                          'P2347','P2924','P3221','P3509','P3827','P4613','P4854','P5250','P5429','P4212','P5513','P508','P4235',
                          'P1807','P2003','P1282','P6766']
print(lista_propiedades)
print(len(lista_propiedades))
lista_propiedades_a_analizar=[]
for prop in lista_propiedades:
    if prop not in lista_a_no_representar:
            lista_propiedades_a_analizar.append(prop)
print(lista_propiedades_a_analizar)
print(len(lista_propiedades_a_analizar))

['P279', 'P407', 'P495', 'P571', 'P580', 'P577', 'P646', 'P674', 'P856', 'P1433', 'P2671', 'P3589', 'P4969', 'P6338', 'P7818', 'P31', 'P50', 'P110', 'P123', 'P136', 'P156', 'P155', 'P179', 'P212', 'P840', 'P910', 'P941', 'P957', 'P1417', 'P1476', 'P2408', 'P3265', 'P3417', 'P5905', 'P17', 'P170', 'P582', 'P1274', 'P1296', 'P1434', 'P3365', 'P4480', 'P127', 'P154', 'P361', 'P393', 'P1889', 'P1984', 'P2638', 'P4087', 'P6839', 'P1080', 'P2031', 'P244', 'P373', 'P527', 'P1424', 'P1448', 'P2003', 'P2013', 'P2163', 'P2354', 'P3984', 'P4073', 'P5357', 'P7859', 'P18', 'P214', 'P227', 'P166', 'P735', 'P7047', 'P21', 'P921', 'P1267', 'P1985', 'P4485', 'P8873', 'P1705', 'P5008', 'P8345', 'P144', 'P747', 'P2047', 'P2318', 'P6175', 'P161', 'P268', 'P6366', 'P58', 'P449', 'P1113', 'P345', 'P364', 'P462', 'P480', 'P905', 'P1258', 'P1265', 'P1545', 'P1970', 'P2334', 'P2465', 'P2508', 'P2509', 'P2529', 'P2603', 'P2704', 'P2747', 'P3107', 'P3143', 'P3138', 'P3212', 'P3302', 'P3704', 'P3868', 'P3933', 'P

In [438]:
#Se obtiene la lista de data properties
lista_data_properties = []
lista_data_properties_id = []
for prop in lista_propiedades_a_analizar:        
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    #Se obtiene el valor booleano del xml string, siendo este "true" en caso afirmativo
    es_OP = string[105:110]
    
    #En caso de que la variable es_OP sea "false", es decir la propiedad sea data propery:
    if es_OP == "false":
        #Se obtienen las subclases de la clase ele
        sparql5 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql5.setQuery("""SELECT  *
        WHERE {
        wd:"""+prop+""" rdfs:label ?label .
        FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
        } 
        LIMIT 1
        """)
        sparql5.setReturnFormat(JSON)
        results = sparql5.query().convert()
        lista_label = []
        for result in results["results"]["bindings"]:
            lista_label.append(result["label"]["value"])
        if(lista_label != []):
            for ele2 in lista_label:
                label = ele2
            label = label + " ("+prop+")"
            lista_data_properties.append(label)
            lista_data_properties_id.append(prop)
print(lista_data_properties)
print(lista_data_properties_id)

['fecha de fundación o creación (P571)', 'fecha de inicio (P580)', 'fecha de publicación (P577)', 'título (P1476)', 'fecha de fin (P582)', 'número de edición (P393)', 'inicio del periodo de actividad (P2031)', 'nombre oficial (P1448)', 'nombre en la lengua nativa (P1705)', 'duración (P2047)', 'número de episodios (P1113)', 'orden dentro de la serie (P1545)', 'número de páginas (P1104)', 'fecha de disolución, retirada o demolición (P576)', 'precio (P2284)', 'nombre del autor (P2093)', 'número (issue) (P433)', 'fecha (P585)', 'nombre en kana (P1814)', 'subtítulo (P1680)', 'fecha representada (P2913)', 'página/s (P304)', 'tomo (P478)', 'masa (P2067)', 'fecha de clausura oficial (P3999)', 'coordenadas (P625)', 'periodicidad (P2896)']
['P571', 'P580', 'P577', 'P1476', 'P582', 'P393', 'P2031', 'P1448', 'P1705', 'P2047', 'P1113', 'P1545', 'P1104', 'P576', 'P2284', 'P2093', 'P433', 'P585', 'P1814', 'P1680', 'P2913', 'P304', 'P478', 'P2067', 'P3999', 'P625', 'P2896']


In [439]:
#Se obtienen el numero de instancias de cada una de las propiedades
sparql5 = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql5.setQuery("""SELECT DISTINCT ?p (COUNT( DISTINCT ?s) as ?sCount)
WHERE
{
?s wdt:P31 wd:"""+clase+""" .
?s ?p ?o .
}
GROUP BY ?p
""")
sparql5.setReturnFormat(JSON)
results = sparql5.query().convert()

lista_numero_propiedades = []
for result in results["results"]["bindings"]:
    if len(result["p"]["value"]) == 33:
        lista_numero_propiedades.append(result["p"]["value"][29:])
    elif len(result["p"]["value"]) == 40:
        lista_numero_propiedades.append(result["p"]["value"][36:])
    elif len(result["p"]["value"]) == 51:
        lista_numero_propiedades.append(result["p"]["value"][47:])
    elif len(result["p"]["value"]) == 34:
        lista_numero_propiedades.append(result["p"]["value"][29:])
    elif len(result["p"]["value"]) == 39:
        lista_numero_propiedades.append(result["p"]["value"][36:])  
    elif len(result["p"]["value"]) == 32:
        lista_numero_propiedades.append(result["p"]["value"][29:]) 
    elif len(result["p"]["value"]) == 41:
        lista_numero_propiedades.append(result["p"]["value"][36:]) 
    elif len(result["p"]["value"]) == 42:
        lista_numero_propiedades.append(result["p"]["value"][35:])
    elif len(result["p"]["value"]) == 52:
        lista_numero_propiedades.append(result["p"]["value"][47:])
    elif len(result["p"]["value"]) == 35:
        lista_numero_propiedades.append(result["p"]["value"][29:])
    elif len(result["p"]["value"]) == 25:
        lista_numero_propiedades.append(result["p"]["value"][19:])
    elif len(result["p"]["value"]) == 30:
        lista_numero_propiedades.append(result["p"]["value"][24:])
    elif len(result["p"]["value"]) == 44:
        lista_numero_propiedades.append(result["p"]["value"][29:])
    elif len(result["p"]["value"]) == 36:
        lista_numero_propiedades.append(result["p"]["value"][29:])
    elif len(result["p"]["value"]) == 52:
        lista_numero_propiedades.append(result["p"]["value"][47:])
    else:
        lista_numero_propiedades.append(result["p"]["value"][34:])
    lista_numero_propiedades.append(int(result["sCount"]["value"]))

print(lista_numero_propiedades)
print(len(lista_numero_propiedades))

['P275', 21, 'P279', 5, 'P276', 1, 'P268', 7, 'P268', 7, 'P287', 1, 'P279', 5, 'P276', 1, 'P275', 21, 'P287', 1, 'P268', 7, 'P304', 3, 'P291', 23, 'P291', 23, 'P304', 3, 'P345', 6, 'P345', 5, 'P373', 14, 'P364', 7, 'P361', 43, 'P373', 14, 'P361', 43, 'P364', 7, 'P393', 11, 'P393', 11, 'P433', 5, 'P437', 1, 'P437', 1, 'P433', 5, 'P407', 1649, 'P407', 1649, 'P449', 1, 'P462', 2, 'P460', 1, 'P478', 3, 'P449', 1, 'P478', 3, 'P462', 2, 'P460', 1, 'P480', 1, 'P488', 1, 'P480', 1, 'P488', 1, 'P527', 8, 'P495', 1327, 'P527', 8, 'P495', 1327, 'P571', 49, 'P571', 49, 'P582', 17, 'P576', 6, 'P580', 360, 'P585', 12, 'P576', 6, 'P577', 411, 'P582', 17, 'P585', 12, 'P577', 411, 'P580', 360, 'P625', 2, 'P629', 2, 'P629', 2, 'P625', 2, 'P641', 1, 'P655', 2, 'P648', 5, 'P648', 5, 'P641', 1, 'P646', 729, 'P646', 729, 'P648', 5, 'P655', 2, 'P676', 1, 'P675', 1, 'P674', 101, 'P646', 729, 'P675', 1, 'P674', 101, 'P676', 1, 'P724', 3, 'P735', 3, 'P724', 3, 'P735', 3, 'P736', 3, 'P750', 1, 'P767', 1, 'P747',

In [440]:
lista_a_representar = []
for data_prop in range(0,len(lista_numero_propiedades)):
    if lista_numero_propiedades[data_prop] in lista_data_properties_id:
        if lista_numero_propiedades[data_prop] not in lista_a_representar:
            lista_a_representar.append(lista_numero_propiedades[data_prop])
            lista_a_representar.append(lista_numero_propiedades[data_prop+1])
print("Lista a ordenar: ", lista_a_representar)

lista_numero_instancias = []
lista_propiedades_representar = []
for ele in range(0,len(lista_a_representar)):
    if ele % 2 != 0:
        lista_numero_instancias.append(lista_a_representar[ele])
    else:
        lista_propiedades_representar.append(lista_a_representar[ele])

lista_final=[]
for ele in range(0,len(lista_numero_instancias)):
    pos_max = lista_numero_instancias.index(max(lista_numero_instancias))
    lista_final.append(lista_propiedades_representar[pos_max])      
    lista_numero_instancias.remove(lista_numero_instancias[pos_max])
    lista_propiedades_representar.remove(lista_propiedades_representar[pos_max])
print("Lista ordenada: ", lista_final)

Lista a ordenar:  ['P304', 3, 'P393', 11, 'P433', 5, 'P478', 3, 'P571', 49, 'P582', 17, 'P576', 6, 'P580', 360, 'P585', 12, 'P577', 411, 'P625', 2, 'P1104', 43, 'P1113', 1, 'P1448', 1, 'P1476', 207, 'P1545', 192, 'P1680', 4, 'P1705', 2, 'P1814', 1, 'P2031', 2, 'P2047', 1, 'P2067', 1, 'P2093', 20, 'P2284', 2, 'P2896', 1, 'P2913', 1, 'P3999', 1]
Lista ordenada:  ['P577', 'P580', 'P1476', 'P1545', 'P571', 'P1104', 'P2093', 'P582', 'P585', 'P393', 'P576', 'P433', 'P1680', 'P304', 'P478', 'P625', 'P1705', 'P2031', 'P2284', 'P1113', 'P1448', 'P1814', 'P2047', 'P2067', 'P2896', 'P2913', 'P3999']


In [441]:
lista_a_meter_en_caja = [lista_final[0],lista_final[1],lista_final[2],lista_final[3],lista_final[4],lista_final[5]]
lista_a_meter_en_caja_label = []
for prop in lista_a_meter_en_caja:
    #Se obtienen las etiquetas
    sparql5 = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql5.setQuery("""SELECT  *
    WHERE {
    wd:"""+prop+""" rdfs:label ?label .
    FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
    } 
    LIMIT 1
    """)
    sparql5.setReturnFormat(JSON)
    results = sparql5.query().convert()
    lista_label = []
    for result in results["results"]["bindings"]:
         lista_label.append(result["label"]["value"])
    if(lista_label != []):
        for ele2 in lista_label:
            label = ele2
        label = label + " ("+prop+")"
        lista_a_meter_en_caja_label.append(label)
print(lista_a_meter_en_caja_label)

['fecha de publicación (P577)', 'fecha de inicio (P580)', 'título (P1476)', 'orden dentro de la serie (P1545)', 'fecha de fundación o creación (P571)', 'número de páginas (P1104)']


In [442]:
a=ET.Element('mxfile')
a.set('type','device')
a.set('etag','d2m_BCRs5NTD1uJDUOON')
a.set('version','13.9.1')
a.set('agent','5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36')
a.set('modified','2020-11-05T11:56:57.085Z')
a.set('host','app.diagrams.net')

b = ET.SubElement(a, 'diagram')
b.set('id','E2t0QyFFZp_JITEdpmB3')

c = ET.SubElement(b, 'mxGraphModel')
c.set('shadow','0')
c.set('math','0')
c.set('pageHeight','1169')
c.set('pageWidth','827')
c.set('pageScale','1')
c.set('page','1')
c.set('fold','1')
c.set('arrows','1')
c.set('connect','1')
c.set('tooltips','1')
c.set('guides','1')
c.set('gridSize','10')
c.set('grid','1')
c.set('dy','18481')
c.set('dx','21749')

d = ET.SubElement(c, 'root')

e = ET.SubElement(d, 'mxCell')
e.set('id','0')
f = ET.SubElement(d, 'mxCell')
f.set('id','1')
f.set('parent','0')

#Caja con la clase inicial
sparql6 = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql6.setQuery("""SELECT  *
WHERE {
wd:"""+clase+""" rdfs:label ?label .
FILTER (langMatches( lang(?label), '"""+idioma+"""' ) )
} 
LIMIT 1
""")
sparql6.setReturnFormat(JSON)
results = sparql6.query().convert()
lista_label = []
for result in results["results"]["bindings"]:
    lista_label.append(result["label"]["value"])
for ele2 in lista_label:
    label = ele2
label = label + " ("+clase+")"
    
g = ET.SubElement(d, 'mxCell')
g.set('id','2')
g.set('parent','1')
g.set('style','rounded=0;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d6b656')
g.set('vertex','1')
g.set('value',label)
g1 = ET.SubElement(g, 'mxGeometry')
g1.set('as','geometry')
g1.set('y','-16200')
g1.set('x','-18500')
g1.set('height','27')
g1.set('width','212')

#Caja con los dataType Strings
aj = ET.SubElement(d,'mxCell')
aj.set('id','3')
aj.set('parent','1')
aj.set('style','rounded=0;whiteSpace=wrap;html=1;align=left;dashed=1')
aj.set('vertex','1')
aj.set('value',lista_a_meter_en_caja_label[0]+'<br>'+lista_a_meter_en_caja_label[1]+'<br>'+lista_a_meter_en_caja_label[2]
      +'<br>'+lista_a_meter_en_caja_label[3]+'<br>'+lista_a_meter_en_caja_label[4]+'<br>'+lista_a_meter_en_caja_label[5])
aj1=ET.SubElement(aj,'mxGeometry')
aj1.set('as','geometry')
aj1.set('y','-16173')
aj1.set('x','-18500')
aj1.set('height','85')
aj1.set('width','212')

x=-20700
y=-16230
iteracion = 0
identificador = 4
for i in range(0,len(lista_varias_clases_una_propiedad)):
    
    #Para las propiedades
    if iteracion % 2 == 0:
        #Si tiene que enlazar con mas de una clase
        if len(lista_varias_clases_una_propiedad[i+1]) > 1:
            #Simbolo de union
            ak = ET.SubElement(d,'mxCell')
            ak.set('id',str(identificador))
            ak.set('parent','1')
            ak.set('style','ellipse;whiteSpace=wrap;html=1;aspect=fixed;fontSize=17;')
            ak.set('vertex','1')
            ak.set('value','<span class="st">U</span>')
            ak1=ET.SubElement(ak,'mxGeometry')
            ak1.set('as','geometry')
            ak1.set('y',str(y+50))
            ak1.set('x',str(x+110))
            ak1.set('height','30')
            ak1.set('width','30')
            
            union = identificador
            identificador = identificador + 1
            
            #Flecha con la propiedad
            bk = ET.SubElement(d, 'mxCell')
            bk.set('id',str(identificador))
            bk.set('parent','1')
            bk.set('target',str(identificador-1))
            bk.set('source','2')
            bk.set('edge','1')
            bk.set('style','edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;')
            bk.set('value',lista_varias_clases_una_propiedad[i])
            bk1 = ET.SubElement(bk, 'mxGeometry')
            bk1.set('as','geometry')
            bk1.set('relative','1')
            
            identificador = identificador + 1

            for ele in lista_varias_clases_una_propiedad[i+1]:
                #Flecha con la propiedad
                ck = ET.SubElement(d, 'mxCell')
                ck.set('id',str(identificador))
                ck.set('parent','1')
                ck.set('target',str(identificador+len(lista_varias_clases_una_propiedad[i+1])))
                ck.set('source',str(union))
                ck.set('edge','1')
                ck.set('style','edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;')
                ck1 = ET.SubElement(ck, 'mxGeometry')
                ck1.set('as','geometry')
                ck1.set('relative','1')
                identificador = identificador + 1
            identificador = identificador - 1
        
        #Si solo tiene que enlazar con una clase
        if len(lista_varias_clases_una_propiedad[i+1]) == 1:
            #Estamos tratando una propiedad
            #Flecha con la propiedad
            k = ET.SubElement(d, 'mxCell')
            k.set('id',str(identificador))
            k.set('parent','1')
            k.set('target',str(identificador+1))
            k.set('source','2')
            k.set('edge','1')
            k.set('style','edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;')
            k.set('value',lista_varias_clases_una_propiedad[i])
            k1 = ET.SubElement(k, 'mxGeometry')
            k1.set('as','geometry')
            k1.set('relative','1')
    
    #Para las clases
    else:
        #Estamos tratando una lista de clases
        num_elem = 1
        for j in lista_varias_clases_una_propiedad[i]:
            if num_elem > 1:
                identificador = identificador +1
            #Caja con clase
            z = ET.SubElement(d, 'mxCell')
            z.set('id',str(identificador))
            z.set('parent','1')
            z.set('style','rounded=0;whiteSpace=wrap;html=1')
            z.set('vertex','1')
            z.set('value',j)
            z1 = ET.SubElement(z, 'mxGeometry')
            z1.set('as','geometry')
            z1.set('y',str(y))
            z1.set('x',str(x))
            z1.set('height','27')
            z1.set('width','212')
            num_elem = num_elem + 1
            x=x+50
            y=y-50
       
    x=x+50
    y=y-50
    iteracion = iteracion + 1
    identificador = identificador + 1

x1 = -18900
y1 = -16020
clase2 = 0
for i in range(0,len(lista_varias_propiedades_una_clase)):
    res = isinstance(lista_varias_propiedades_una_clase[i], str) 
    #Si son propiedades
    if str(res) == "True":
        #Flecha con la propiedad
        bk = ET.SubElement(d, 'mxCell')
        bk.set('id',str(identificador))
        bk.set('parent','1')
        bk.set('target',str(clase2))
        bk.set('source','2')
        bk.set('edge','1')
        bk.set('style','edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;')
        bk.set('value',str(lista_varias_propiedades_una_clase[i]))
        bk1 = ET.SubElement(bk, 'mxGeometry')
        bk1.set('as','geometry')
        bk1.set('relative','1')
        identificador = identificador + 1
    #Si son clases
    else:
        #Caja con la clase
        z = ET.SubElement(d, 'mxCell')
        z.set('id',str(identificador))
        z.set('parent','1')
        z.set('style','rounded=0;whiteSpace=wrap;html=1')
        z.set('vertex','1')
        z.set('value',str(lista_varias_propiedades_una_clase[i][0]))
        z1 = ET.SubElement(z, 'mxGeometry')
        z1.set('as','geometry')
        z1.set('y',str(y1))
        z1.set('x',str(x1))
        z1.set('height','27')
        z1.set('width','212')
        clase2 = identificador
        identificador = identificador + 1
        x1 = x1 - 50
        y1 = y1 + 50
ET.dump(a)
tree = ET.ElementTree(a)
tree.write("output_final.xml", encoding='utf-8', xml_declaration=True)                       

<mxfile agent="5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36" etag="d2m_BCRs5NTD1uJDUOON" host="app.diagrams.net" modified="2020-11-05T11:56:57.085Z" type="device" version="13.9.1"><diagram id="E2t0QyFFZp_JITEdpmB3"><mxGraphModel arrows="1" connect="1" dx="21749" dy="18481" fold="1" grid="1" gridSize="10" guides="1" math="0" page="1" pageHeight="1169" pageScale="1" pageWidth="827" shadow="0" tooltips="1"><root><mxCell id="0" /><mxCell id="1" parent="0" /><mxCell id="2" parent="1" style="rounded=0;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d6b656" value="historieta (Q1004)" vertex="1"><mxGeometry as="geometry" height="27" width="212" x="-18500" y="-16200" /></mxCell><mxCell id="3" parent="1" style="rounded=0;whiteSpace=wrap;html=1;align=left;dashed=1" value="fecha de publicación (P577)&lt;br&gt;fecha de inicio (P580)&lt;br&gt;título (P1476)&lt;br&gt;orden dentro de la serie (P1545)&lt;br&gt;fecha de fundación o crea